<a href="https://colab.research.google.com/github/Winston68432/streamlit-example/blob/master/NBA_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:


!pip install -q streamlit




In [84]:

!pip install -q streamlit_option_menu 

In [111]:
%%writefile app.py
import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

df_nba_complete = pd.read_csv("NBA Shot Locations 1997 - 2020.csv")
df_player = pd.read_csv("player_data.csv",sep = ',')

# reduction aux 20 joueurs
names = ['Tim Duncan','Kobe Bryant','Allen Iverson','Steve Nash','Ray Allen','Paul Pierce','Pau Gasol','Tony Parker','Manu Ginobili','Dwyane Wade',
   'LeBron James','Chris Paul','Kevin Durant','Russell Westbrook','James Harden','Stephen Curry','Kawhi Leonard','Damian Lillard','Anthony Davis','Giannis Antetokounmpo']
df_nba=df_nba_complete[df_nba_complete['Player Name'].isin(names)]
# st.image("nba_project.PNG")

#Merge avec df_player
df_nba=df_nba.merge(df_player,how='left',left_on="Player Name", right_on="name")

#Ajoute shottime
df_nba.loc[df_nba['Period'] < 5, 'Shot_Time'] = (720-(60 * df_nba['Minutes Remaining'] + df_nba['Seconds Remaining']) + (df_nba['Period']-1)*12*60)
df_nba.loc[df_nba['Period'] > 4, 'Shot_Time'] = (300-(60 * df_nba['Minutes Remaining'] +df_nba['Seconds Remaining']) + ((df_nba['Period']-5)*5+48)*60)

df_nba['Shot_Time'] = df_nba['Shot_Time'].round(decimals=0).astype(int)

#clustering en fonction des actions réussis
df_nba.loc[df_nba['Shot Made Flag'] ==1, 'Success'] = "yes"
df_nba.loc[df_nba['Shot Made Flag'] ==0, 'Success'] = "no"
df =pd.crosstab(df_nba['Action Type'],df_nba['Success'],normalize = "index").reset_index()

df.loc[df["yes"] > 0,'CLASSE'] = "1"
df.loc[df["yes"] > 0.10,'CLASSE']="2"
df.loc[df["yes"] > 0.20,'CLASSE']="3"
df.loc[df["yes"] > 0.30,'CLASSE']="4"
df.loc[df["yes"] > 0.40,'CLASSE']="5"
df.loc[df["yes"] > 0.50,'CLASSE'] = "6"
df.loc[df["yes"] > 0.60,'CLASSE'] = "7"
df.loc[df["yes"] > 0.70 ,'CLASSE'] = "8"
df.loc[df["yes"] > 0.80 ,'CLASSE'] = "9"
df.loc[df["yes"] > 0.90 ,'CLASSE'] = "10"
df_nba=df_nba.merge(df,how='left')

#VAriables categorielles
df_nba=df_nba.join(pd.get_dummies(df_nba['Shot Zone Basic']))
df_nba=df_nba.join(pd.get_dummies(df_nba['Shot Zone Area']))
df_nba=df_nba.join(pd.get_dummies(df_nba['Shot Zone Range']))
df_nba=df_nba.join(pd.get_dummies(df_nba['position']))

#1 joueur
names = ['Allen Iverson']   
df_nba_player=df_nba[df_nba['Player Name'].isin(names)]

to_drop = ['Game ID', 'Game Event ID', 'Player ID', 'Player Name',
           'Team ID', 'Team Name','Period', 'Minutes Remaining', 'Seconds Remaining', 
           'Action Type', 'Shot Type', 'Shot Zone Basic','Shot Zone Area', 'Shot Zone Range',
           'Shot Made Flag', 'Game Date', 'Home Team','Shot Distance','X Location','Y Location',  'Away Team', 'Season Type',
           'name', 'year_start', 'year_end','height','weight','birth_date',  'college', 'Success','no', 'yes','position']
data = df_nba_player.drop(to_drop,axis=1)
target=df_nba_player['Shot Made Flag']



# 1. as sidebar menu
with st.sidebar:
    selected = option_menu("Menu", ["Introduction", "Dataset","Analyse Descriptive", "Creation des Variables","Modelisation","Données Manquantes", "Exploration","Conclusion"], 
        icons=['house','123','image','keyboard','graph-up', 'gear','alarm','check'], menu_icon="cast", default_index=0)

#Page Introduction 

if selected=="Introduction":
  header=st.container()
  with header:
      st.subheader('*Introduction*')
      st.write("L’immense popularité des sports d’équipe aux Etats Unis et du Basket Ball par exemple, ont favorisé le développement de la collection de données sur ces sports.  Nous disposons donc de données extrêmement riches, collectées sur tous les aspects du sport, sur les joueurs, les matchs de Basket Ball, les tirs marqués ou non, et toutes les circonstances particulières dans lesquelles les tirs ont été effectués.") 
      st.write("En considérant les 20 meilleurs joueurs NBA du 21ème siècle selon la chaine de sport ESPN, nous allons analyser les tirs effectués par ces 20 joueurs, et développer des modèles prédictifs de Machine Learning sur ces tirs..")  

#Page Dataset

if selected=="Dataset":

  st.write('Pour notre étude, nous nous sommes essentiellement basés pour nos recherches sur le dataset des tirs NBA entre 1997 et 2020 (nba-shot-locations) ainsi que sur le dataset nommé players_data. Ces 2 datasets peuvent être téléchargés via le site Kaggle.')
  
  headerdataset1=st.container()
  headerdataset2=st.container()
  
  with headerdataset1:
      st.subheader('*1 )DATASET NBA SHOTS LOCATIONS*')
      st.write("Ce dataset recense tous les tirs des joueurs de la NBA de 1997 à 2000.") 
      st.write("Seuls les tirs à 2 points et 3 points sont mentionnés. On n’étudiera donc pas la précision des joueurs aux lancers francs.") 
      st.write("La variable **Shot Made Flag*** sera notre variable cible qui déterminera la réussite ou l’échec d’un tir.")
      st.write(df_nba_complete.head())
      st.write("")
      st.write("**On va restreindre par la suite ce dataset à 20 joueurs pour la modélisation.**")  
  with headerdataset2:
      st.subheader('*2) DATASET PLAYER DATA*')
      st.write("Ce dataset indique pour chaque joueur son nom, sa date de naissance, son poids, sa taille, son collège d’étude, son début et sa fin de carrière ainsi que sa position sur le terrain. ") 
      st.write(df_player.head())  

#Page Analyse Descriptive

if selected=="Analyse Descriptive":
      st.write("blabla")

#Page Analyse Descriptive

if selected=="Creation des Variables":
     
  headervariable1=st.container()
  headervariable2=st.container()
  headervariable3=st.container() 
  headervariable4=st.container()    
  headervariable5=st.container() 

  with headervariable1:   
      st.subheader('*1) Shot TIME*') 
      st.write("On a démontré que le pourcentage de réussite baissait en fonction des périodes. Cela se confirme pour nos 20 joueurs.")
      st.image("pourcentage de réussite.PNG",width=400)
      st.write("On a donc décidé d’enrichir notre data set avec la variable Shot_Time qui permet de savoir quand le shot a été effectué en tenant compte de la période, et du temps restant.")
           
      Periode=st.selectbox('**Choisissez la période**: ', options=[1,2,3,4,5,6,7])     
      Minutes=st.slider('**Choisissez les minutes restantes** : ', min_value=0,max_value=11) 
      Seconde=st.slider('**Choisissez les secondes restantes** : ', min_value=0,max_value=59) 
    
      if Periode <5:
          Shot_Time=(720-(60*Minutes+Seconde)+(Periode-1)*12*60)
      else:
          Shot_Time=(300-(60*Minutes+Seconde)+((Periode-5)*5+48)*60)

      col1, col2, col3 = st.columns([1,1,1])

      with col1:
        
          st.subheader("**Le shot a été effectué à la**") 
      with col2:
          st.subheader(Shot_Time)
      with col3:  
          st.subheader("**seconde du match**") 
      st.write("")
      st.write("")  

  with headervariable2:   
      st.subheader('*2) GROUPEMENT DES ACTIONS*') 
      st.write("L’analyse descriptive du dataframe principale nous a montré qu’il y avait 70 types d’actions")
      st.write("Ce nombre d’actions est beaucoup trop important .")  
      st.write("On a cherché une méthode de regroupement empirique. ")
      st.write("L’idée a été de calculer le pourcentage de réussite de ces actions à l’aide d’un crosstab puis de créer 10 classes en fonction de ces pourcentages. Dans la classe 10, on met toutes les actions qui ont un pourcentage de réussite supérieur à 90, puis dans la classe 9 celles qui ont un pourcentage de réussite compris entre 80 et 90. ")
      st.write("**On est donc passé de 70 actions à 10 classes**")
      st.write("")
      st.write("") 
  with headervariable3:   
      st.subheader('*3)	POSTE DES JOUEURS*') 
      st.write("Nous avons aussi ajouté au   data set principal le poste des joueurs. Pour cela nous avons lié notre data frame au data frame « player_data.csv » qui contient le poste des joueurs.")    
      st.write("")
      st.write("") 
  with headervariable4: 
      st.subheader('*4) VARIABLES CATEGORIELLES*') 
      st.write("Afin d’exploiter les variables catégorielles nous avons utilisé la méthode panda.get.dummies de python.")  
      st.write("Cette transformation a été appliquée au variables suivantes : ")
      st.write("*-Shot Zone Basic*" )  
      st.write("*-Shot Zone Area*" ) 
      st.write("*-Shot Zone Range*" ) 
      st.write("*-position*" )
      st.write("")
      st.write("")   
  with headervariable5: 
      st.subheader('*5) DATAFRAME FINAL*') 
      st.write("Nous avons évidemment enlevé toutes les colonnes id, toutes les colonnes catégorielles que nous avions transformées avec la méthode get.dummies. .")  
      st.write("Nous n’avons pas conservé  des colonnes que nous jugions inutiles.  Par exemple, nous avons enlevé le collège du joueur qui n’influence pas son taux de réussite.")
      st.write( "Ci-dessous un exemple des deux dataframe qui serviront  ")


#Page Exploration

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Titre
st.title("Le coefficient de Gini appliqué au basket ball")

#En-tête
st.header("La courbe de Lorenz")

#Ecrire texte
st.write("La courbe de Lorenz est une représentation graphique de la distribution d'une variable (revenu, patrimoine, etc.) au sein d'une population. Elle permet de calculer le coefficient de Gini, qui mesure l'inégalité de cette variable dans une population donnée. Dans notre cas, le but est de vérifier si la densité des joueurs au sein d'une équipe est homogène ou hétérogène.")

# Analyse de deux équipes

#En-tête
st.header("Deux équipes analysées: Les San Antonio Spurs et les Houston Rockets")

#Ecrire texte
st.write("Les San Antonio Spurs tendent à être plus collectif, se reposant sur un trio de joueurs performants en terme de moyenne de nombre de points tandis que les Houston Rockets est une équipe qui se repose plus sur les compétences d'un seul joueur")



#Moyenne du nombre de points de l'équipe des San Antonio Spurs de 2000 à 2010

#En-tête
st.header("Moyenne du nombre de points de l'équipe des San Antonio Spurs de 2000 à 2010")

#Sous en-tête
st.subheader("Moyenne du nombre de points de l'équipe des San Antonio Spurs pour la saison 2000/2001")


df1 = pd.DataFrame(columns=['Joueur San Antonio Spurs','PTS','Cpl','CPTS','Cpl%','CPTS%'], index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])


df1.loc[1] = pd.Series({'Joueur San Antonio Spurs':'Shawnelle Scott','PTS':1.6,'Cpl':1})
df1.loc[2] = pd.Series({'Joueur San Antonio Spurs':'Ira Newble','PTS':2,'Cpl':2})
df1.loc[3] = pd.Series({'Joueur San Antonio Spurs':'Jaren Jackson','PTS':2.4,'Cpl':3})
df1.loc[4] = pd.Series({'Joueur San Antonio Spurs':'Derrick Dial','PTS':2.6,'Cpl':4})
df1.loc[5] = pd.Series({'Joueur San Antonio Spurs':'Steve Kerr','PTS':3.3,'Cpl':5})
df1.loc[6] = pd.Series({'Joueur San Antonio Spurs':'Samaki Walker','PTS':5.3,'Cpl':6})
df1.loc[7] = pd.Series({'Joueur San Antonio Spurs':'Danny Ferry','PTS':5.6,'Cpl':7})
df1.loc[8] = pd.Series({'Joueur San Antonio Spurs':'Avery Johnson','PTS':5.6,'Cpl':8})
df1.loc[9] = pd.Series({'Joueur San Antonio Spurs':'Terry Porter','PTS':7.2,'Cpl':9})
df1.loc[10] = pd.Series({'Joueur San Antonio Spurs':'Malik Rose','PTS':7.7,'Cpl':10})
df1.loc[11] = pd.Series({'Joueur San Antonio Spurs':'Sean Elliott','PTS':7.9,'Cpl':11})
df1.loc[12] = pd.Series({'Joueur San Antonio Spurs':'Antonio Daniels','PTS':9.4,'Cpl':12})
df1.loc[13] = pd.Series({'Joueur San Antonio Spurs':'David Robinson','PTS':14.4,'Cpl':13})
df1.loc[14] = pd.Series({'Joueur San Antonio Spurs':'Derek Anderson','PTS':15.5,'Cpl':14})
df1.loc[15] = pd.Series({'Joueur San Antonio Spurs':'Tim Duncan','PTS':22.2,'Cpl':15})

df1["CPTS"] = np.cumsum(df1["PTS"])

df1["Cpl%"] = df1["Cpl"].apply(lambda x : ((x/df1.loc[len(df1),"Cpl"])*100))

df1["CPTS%"] = df1["CPTS"].apply(lambda x : (x/df1.loc[len(df1),"CPTS"])*100)

df1

#df1 = pd.read_csv("football_data.csv")
if st.checkbox('Show dataframe'):
    st.write(df1)

#Sous en-tête
st.subheader("Courbe de Lorenz et coefficient de Gini des San Antonio Spurs pour la saison 2000/2001")





# Constitution des DataFrames

st.header("Constitution des DataFrames")

#Sous en-tête
st.subheader("Nomination des colonnes")


#Ecrire texte
st.write("Gini: Coefficient qui détermine l’homogénéité ou l’hétérogénéité d’une équipe sportive")

st.write("1_Marqueur : Le meilleur marqueur de l’équipe en question")

st.write("Moyenne_PTS_1_marqueur : Moyenne de points du meilleur marqueur")

st.write("Gini_sans_1_marqueur : Coefficient Gini de toute l’équipe sans le meilleur marqueur. Permet de voir si le taulier de l’équipe la « déséquilibre» en terme de moyenne de points.")
st.write("Plus l’écart sera significatif avec le coefficient Gini de l’équipe, plus l’équipe dépend du meilleur marqueur")
st.write("Par rapport à notre étude, un écart supérieur à 0.3 signifie que l’écart commence à être conséquent")

st.write("Gini_des_trois_meilleurs_marqueurs: Coefficients Gini des trois meilleurs marqueurs de l’équipe.Permet de voir l’homogénéité ou l’hétérogénéité des trois meilleurs marqueurs de l’équipe en terme de moyenne de points.")
st.write("Plus ce nombre se rapproche de zéro, plus la moyenne des points sera correctement répartie entre les trois joueurs.")
st.write("Par rapport à notre étude, un coefficient supérieur à 1 ou qui se rapproche très fortement de 1 signifie un début d’hétérogénéité.")

st.write("2_marqueur: Le deuxième meilleur marqueur de l’équipe")

st.write("3_marqueur: Le troisième meilleur marqueur de l’équipe")


#Sous en-tête

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


st.subheader("Les quatre DataFrames de notre étude")

st.subheader("DataFrame des San Antonio Spurs des années 2000/2010")

df4 = pd.read_csv('DataFrame_San_Antonio_Spurs_2000_2010.csv')

df4


st.subheader("DataFrame des San Antonio Spurs des années 2010/2018")

df5 = pd.read_csv('DataFrame_San_Antonio_Spurs_2010_2018.csv')

df5



st.subheader("DataFrame des Houston Rockets des années 2000/2010")

df6 = pd.read_csv('DataFrame_Houston_Rockets_2000_2010.csv')

df6



st.subheader("DataFrame des Houston Rockets des années 2010/2018")

df7 = pd.read_csv('DataFrame_Houston_Rockets_2010_2018.csv')

df7


st.subheader("coefficients de Gini de l’équipe des San Antonio Spurs et des Houston Rockets de 2000 à 2010")

plt.figure(figsize=(10, 6))
barWidth = 0.25

x1 = range(10)

x2 = [r + barWidth for r in range(10)]


plt.bar(x1, df4["Gini"], width = 0.2, label = 'Gini Spurs')
plt.bar(x2, df6["Gini"], width = 0.2, label = 'Gini Rockets')

plt.xticks(rotation='vertical')
plt.xticks([0,1,2,3,4,5,6,7,8,9], ['2000/2001', '2001/2002', '2002/2003', '2003/2004', '2004/2005', '2005/2006', '2006/2007', '2007/2008', '2008/2009', '2009/2010'])
plt.legend()
st.pyplot(plt.figure(figsize=(10, 6)));














   


Overwriting app.py


# Nouvelle section

In [114]:
!streamlit run app.py &>/dev/null&

In [116]:
!pip install pyngrok==4.1.1
from pyngrok import ngrok
!ngrok authtoken 2Kd7Oc6LGJ1aEzPuHlyPZP0uuIP_3QYAYHtWf1H9RXtmP3aDJ
public_url = ngrok.connect(port='8501')
print(public_url)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
http://8aac-35-201-235-237.ngrok.io


In [115]:
ngrok.kill()